In [1]:
import os
import numpy as np
import pandas as pd
import seaborn
# --- Sanity-check plot: seasonality across counties ---
import matplotlib.pyplot as plt
import seaborn as sns



PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_RAW = os.path.join(PROJECT_ROOT, "data", "raw")
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
SCOPE = "dmv"

#PM25_DAILY_PATH = os.path.join(DATA_RAW, "pm25_daily_dc_md.csv")
#df_pm = pd.read_csv(PM25_DAILY_PATH, low_memory=False)

#print("Loaded:", df_pm.shape)
#print([c for c in df_pm.columns if "state" in c.lower() or "county" in c.lower()][:50])
#df_pm.head()
PM25_RAW_FILES = [
    os.path.join(DATA_RAW, "./aqs/daily_88101_2018.csv"),
    os.path.join(DATA_RAW, "./aqs/daily_88101_2019.csv"),
    os.path.join(DATA_RAW, "./aqs/daily_88101_2022.csv"),
    os.path.join(DATA_RAW, "./aqs/daily_88101_2023.csv"),
]

In [2]:
dfs = []
for fp in PM25_RAW_FILES:
    if os.path.exists(fp):
        dfs.append(pd.read_csv(fp, low_memory=False))
    else:
        print("Missing:", fp)
if len(dfs) == 0:
    raise FileNotFoundError("No PM2.5 raw files found. Check PM25_RAW_FILES paths.")

df = pd.concat(dfs, ignore_index=True)
print("Loaded raw AQS:", df.shape)
print("Columns:", list(df.columns)[:25])


Loaded raw AQS: (2921381, 29)
Columns: ['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration', 'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type', 'Observation Count', 'Observation Percent', 'Arithmetic Mean', '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name', 'Local Site Name', 'Address', 'State Name']


In [3]:
# --- Build GEOID from AQS codes (works for both scopes) ---
df["state_fips"] = df["State Code"].astype(str).str.zfill(2)
df["county_fips"] = df["County Code"].astype(str).str.zfill(3)
df["geoid"] = df["state_fips"] + df["county_fips"]

if SCOPE == "dmv":
    dmv_geoids = {
        "11001","24005","24510","24015","24019","24023",
        "24025","24027","24029","24031","24033","24043"
    }
    df = df[df["geoid"].isin(dmv_geoids)].copy()
elif SCOPE == "us":
    pass
else:
    raise ValueError("SCOPE must be 'dmv' or 'us'")


In [4]:
# Parse date
df["date"] = pd.to_datetime(df["Date Local"], errors="coerce")
df = df.dropna(subset=["date", "geoid"])

# Keep PM2.5 daily measurement column
y_col = "Arithmetic Mean"
df = df.dropna(subset=[y_col])

# Month column
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()


In [5]:
county_month = (
    df.groupby(["geoid", "month"])
      .agg(
          pm25_median=(y_col, "median"),
          pm25_mean=(y_col, "mean"),
          n_obs=(y_col, "size"),
      )
      .reset_index()
)

print("county_month:", county_month.shape)
county_month.head()


county_month: (287, 5)


,geoid,month,pm25_median,pm25_mean,n_obs
0,11001,2018-01-01,8.9000,10.115170,249
1,11001,2018-02-01,8.7000,9.349745,230
2,11001,2018-03-01,7.1000,7.949958,258
3,11001,2018-04-01,7.1125,7.824540,250
4,11001,2018-05-01,9.6000,10.034525,257


In [6]:
out_name = "pm25_county_monthly_dmv.csv" if SCOPE == "dmv" else "pm25_county_monthly_us.csv"
OUT_PATH = os.path.join(DATA_PROCESSED, out_name)

county_month.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


Saved: /Users/aishwaryaiyer/Documents/GitHub/project-air-health/data/processed/pm25_county_monthly_dmv.csv
